In [ ]:
# Part 1
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import micropip
await micropip.install("seaborn")
import seaborn as sns
import seaborn as sns
await micropip.install("plotly")
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import os
from datetime import datetime
import duckdb

warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

Matplotlib is building the font cache; this may take a moment.


In [ ]:
# Load NYC 311 sample data
#reads CSV lazily and runs SQL on it without loading everything
query = """
SELECT strftime('%Y-%m', "Created Date") AS month, COUNT(*) AS cnt
FROM read_csv_auto('nyc_311_sample.csv', header=True)
GROUP BY month
ORDER BY month
"""
df = duckdb.query(query).to_df()
print(df.head(10))

In [ ]:
#check duplicates
num_dupes = df.duplicated().sum()
print(f"Number of duplicate rows: {num_dupes}")

In [ ]:
#check missing values and clean them in part 2

In [ ]:
# Part 2
# Basic Data Profiling: Analyze service request status distribution
status_counts = df['Status'].value_counts()
for status, count in status_counts.head().items():
    percentage = (count / len(df)) * 100
    print(f"  {status}: {count:,} ({percentage:.1f}%)")

In [ ]:
# Missing Value Analysis:
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})
missing_analysis = missing_analysis.sort_values('Missing_Percentage', ascending=False)
print("Top 10 columns with missing values:")
top_missing = missing_analysis[missing_analysis['Missing_Count'] > 0].head(10)
for _, row in top_missing.iterrows():
    print(f"  {row['Column']}: {row['Missing_Count']:,} ({row['Missing_Percentage']:.1f}%)")



In [ ]:
# Drop those columns with high percentage of missing values
high_missing_cols = missing_analysis.loc[
    missing_analysis["Missing_Percentage"] > 90, "Column"
]

df = df.drop(columns=high_missing_cols)

In [ ]:
# Complaint Type Analysis: Identify most common service request types
top_complaints = df['Complaint Type'].value_counts().head(15)
print("Top 15 Complaint Types:")
for i, (complaint, count) in enumerate(top_complaints.items(), 1):
    percentage = (count / len(df)) * 100
    print(f"{i:2d}. {complaint}: {count:,} ({percentage:.1f}%)")

# Cross-analysis: Top complaint types by government agency
print(f"\nComplaint Types by Agency (Top 5 agencies):")
top_agencies = df['Agency Name'].value_counts().head(5).index
for agency in top_agencies:
    agency_data = df[df['Agency Name'] == agency]
    top_agency_complaints = agency_data['Complaint Type'].value_counts().head(3)
    print(f"\n{agency}:")
    for complaint, count in top_agency_complaints.items():
        pct = (count / len(agency_data)) * 100
        print(f"   • {complaint}: {count:,} ({pct:.1f}%)")

In [ ]:
# Trend Analysis
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Year'] = df['Created Date'].dt.year
df['Month'] = df['Created Date'].dt.month
df['DayOfWeek'] = df['Created Date'].dt.day_name()
df['Hour'] = df['Created Date'].dt.hour
df['YearMonth'] = df['Created Date'].dt.to_period('M')

# Analyze yearly distribution
print("Requests by Year:")
yearly_counts = df['Year'].value_counts().sort_index()
for year, count in yearly_counts.items():
    print(f"  {year}: {count:,}")

# Analyze seasonal patterns (monthly distribution)
print("\nRequests by Month (average):")
monthly_avg = df.groupby('Month').size()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month, count in monthly_avg.items():
    month_name = month_names[month-1]
    print(f"  {month_name}: {count:,}")

# Analyze weekly patterns
print("\nRequests by Day of Week:")
dow_counts = df['DayOfWeek'].value_counts()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for day in day_order:
    if day in dow_counts:
        count = dow_counts[day]
        pct = (count / len(df)) * 100
        print(f"  {day}: {count:,} ({pct:.1f}%)")

In [ ]:
# Requests distribution and per-capita analysis
print(" Requests by Borough:")
borough_stats = df['Borough'].value_counts()
for borough, count in borough_stats.items():
    if pd.notna(borough):
        percentage = (count / len(df)) * 100
        print(f"  {borough}: {count:,} ({percentage:.1f}%)")

# Most common complaint type per borough
print(f"\nTop Complaint Type by Borough:")
for borough in borough_stats.index[:5]:
    if pd.notna(borough):
        borough_data = df[df['Borough'] == borough]
        top_complaint = borough_data['Complaint Type'].value_counts().head(1)
        if not top_complaint.empty:
            complaint = top_complaint.index[0]
            count = top_complaint.iloc[0]
            pct = (count / len(borough_data)) * 100
            print(f"  {borough}: {complaint} ({count:,}, {pct:.1f}%)")

# Per capita analysis using 2020 Census data
print(f"\nRequests Per Capita by Borough (Estimated):")
borough_population = {
    'BROOKLYN': 2736074,
    'QUEENS': 2405464,
    'MANHATTAN': 1694251,
    'BRONX': 1472654,
    'STATEN ISLAND': 495747
}
print("Borough population estimates used (2020 Census):")
for borough in borough_stats.index[:5]:
    if pd.notna(borough) and borough in borough_population:
        requests = borough_stats[borough]
        population = borough_population[borough]
        per_capita = (requests / population) * 1000  # per 1,000 residents
        print(f"  {borough}: {per_capita:.2f} requests per 1,000 residents")
        print(f"    (Total: {requests:,} requests, Population: {population:,})")
    elif pd.notna(borough):
        requests = borough_stats[borough]
        print(f"  {borough}: {requests:,} requests (population data not available)")

In [ ]:
# Chart 1: Top Complaint Types Bar Chart
plt.figure(figsize=(12, 8))
top_15_complaints = df['Complaint Type'].value_counts().head(15)
plt.barh(range(len(top_15_complaints)), top_15_complaints.values, 
         color=plt.cm.viridis(np.linspace(0, 1, len(top_15_complaints))))
plt.yticks(range(len(top_15_complaints)), top_15_complaints.index)
plt.xlabel('Number of Requests')
plt.title('Top 15 Complaint Types in NYC 311 System', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()

# Add value labels to bars
for i, v in enumerate(top_15_complaints.values):
    plt.text(v + max(top_15_complaints.values) * 0.01, i, f'{v:,}', 
             va='center', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Chart 2: Monthly Pattern
plt.figure(figsize=(8, 5))
monthly_counts = df.groupby('Month').size()
plt.plot(monthly_counts.index, monthly_counts.values, marker='o', linewidth=2, markersize=8)
plt.title('Requests by Month', fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Requests')
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13), 
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

In [ ]:
# Chart 3: Weekly Pattern
plt.figure(figsize=(8, 5))
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = df['DayOfWeek'].value_counts().reindex(dow_order)
plt.bar(range(len(dow_counts)), dow_counts.values, color=plt.cm.Set3(range(len(dow_counts))))
plt.title('Requests by Day of Week', fontweight='bold')
plt.xlabel('Day of Week')
plt.ylabel('Number of Requests')
plt.xticks(range(len(dow_counts)), [day[:3] for day in dow_order], rotation=45)
plt.show()

In [ ]:
# Chart 4: Hourly Pattern
plt.figure(figsize=(8, 5))
hourly_counts = df['Hour'].value_counts().sort_index()
plt.plot(hourly_counts.index, hourly_counts.values, marker='o', linewidth=2, markersize=6)
plt.title('Requests by Hour of Day', fontweight='bold')
plt.xlabel('Hour')
plt.ylabel('Number of Requests')
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 24, 4))
plt.show()

In [ ]:
# Key Insights:
print("KEY FINDINGS:")
total_requests = len(df)
date_span = (df['Created Date'].max() - df['Created Date'].min()).days

# Dataset overview metrics
print(f"\nDATASET OVERVIEW:")
print(f"  • Total Service Requests: {total_requests:,}")
print(f"  • Time Period: {date_span} days ({df['Created Date'].min().strftime('%Y-%m-%d')} to {df['Created Date'].max().strftime('%Y-%m-%d')})")
print(f"  • Average Daily Requests: {total_requests/date_span:.0f}")

# Service request patterns
top_complaint = df['Complaint Type'].value_counts().index[0]
top_complaint_pct = (df['Complaint Type'].value_counts().iloc[0] / len(df)) * 100
print(f"\nCOMPLAINT INSIGHTS:")
print(f"  • Most Common Complaint: {top_complaint} ({top_complaint_pct:.1f}%)")
print(f"  • Total Complaint Types: {df['Complaint Type'].nunique()}")

# Seasonal pattern analysis
busiest_month = monthly_avg.idxmax()
busiest_day = dow_counts.idxmax()
busiest_hour = hourly_counts.idxmax()
print(f"\nTEMPORAL PATTERNS:")
print(f"  • Busiest Month: {month_names[busiest_month-1]} (month #{busiest_month})")
print(f"  • Busiest Day: {busiest_day}")
print(f"  • Peak Hour: {busiest_hour}:00")

# Geographic distribution insights
busiest_borough = borough_stats.index[0]
busiest_borough_pct = (borough_stats.iloc[0] / len(df)) * 100
print(f"\n GEOGRAPHIC PATTERNS:")
print(f"  • Busiest Borough: {busiest_borough} ({busiest_borough_pct:.1f}%)")
print(f"  • Total Boroughs: {df['Borough'].nunique()}")
